In [ ]:
import os
import pandas as pd
import pyarrow.parquet as pq
import re
import json

In [ ]:
parquet_file_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data/Ecogeographic survey of the target species-Costa Rica-2004.parquet'
df = pd.read_parquet(parquet_file_path, engine='pyarrow')
df.head()

In [ ]:
#Contar caracteres de cada pagina extraida
df['conteo_caracteres'] = df['Texto_Extraido'].astype(str).apply(len)
df.head()

In [ ]:
# Obtener número de pagina
def extract_page_number(image_name):
    match = re.search(r'\d+', image_name)
    if match:
        return int(match.group())
    return 0

In [ ]:
# Crear nueva columna "Numero_Pagina"
df['Numero_Pagina'] = df['Nombre_Imagen'].apply(extract_page_number)
# Ordenar df por el número de página de menor a mayor
df_sorted = df.sort_values(by='Numero_Pagina')
df_sorted.head()

In [ ]:
def process_parquet(carpeta_parquet, columna_texto, columna_imagen, nueva_columna_caracteres='conteo_caracteres', nueva_columna_pagina='Numero_Pagina'):
    def extract_page_number(image_name):
        # Extraer el número de página
        match = re.search(r'\d+', image_name)
        if match:
            return int(match.group())
        return 0

    # Lista de archivos Parquet en la carpeta
    archivos_parquet = [f for f in os.listdir(carpeta_parquet) if f.endswith('.parquet')]

    for archivo in archivos_parquet:
        # Leer el archivo Parquet en un DataFrame
        path_archivo = os.path.join(carpeta_parquet, archivo)
        df = pd.read_parquet(path_archivo)

        # Verificar si las columnas especificadas existen en el DataFrame
        if columna_texto in df.columns and columna_imagen in df.columns:
            # Castear las columnas de texto como object
            df[columna_texto] = df[columna_texto].astype('object')
            df[columna_imagen] = df[columna_imagen].astype('object')
            
            # Contar caracteres en la columna de texto
            df[nueva_columna_caracteres] = df[columna_texto].astype(str).apply(len)
            
            # Crear una nueva columna con el número de página
            df[nueva_columna_pagina] = df[columna_imagen].apply(extract_page_number)
            
            # Ordenar el DataFrame por el número de página de menor a mayor
            df_sorted = df.sort_values(by=nueva_columna_pagina)
            
            # Guardar el DataFrame modificado de nuevo como archivo Parquet
            df_sorted.to_parquet(path_archivo, index=False, engine='pyarrow')
        else:
            print(f"Las columnas '{columna_texto}' y/o '{columna_imagen}' no se encuentran en el archivo {archivo}")

In [ ]:
carpeta_parquet = '/home/ec2-user/SageMaker/genebanks/data/3_out_data'
columna_texto = 'Texto_Extraido'
columna_imagen = 'Nombre_Imagen'
process_parquet(carpeta_parquet, columna_texto, columna_imagen)